In [1]:
%config Completer.use_jedi = False

In [2]:
import dash_html_components as html
import dash_bootstrap_components as dbc 
import dash_core_components as dcc 

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

from jupyter_dash import JupyterDash
from dash.dependencies import Input, State, Output

import pandas as pd 
import numpy as np

import random

import warnings
warnings.filterwarnings('ignore')

In [3]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.PULSE])

app.title = 'Speed Dating'

In [4]:
prep_list=[
           'After downloading the data collected by Ray Fisman and Sheena Iyengar, an economist and a psychologist, \
            looking through the variables, if more than half the data was missing, I chose to ignore it',
           'With the data I got, the important variable is "Match". So I got just the variables with a noticeable correlation to the "Match" variable',
           'For easier manipulation, I saved the new data in a file',
           'Just for a preview, I made a radial plot of the averages of the variables'
          ]

mod_list = [
            'I started by splitting the new data into train and test sub datasets',
            'Making a correlation plot, I noticed a high correlation with dec_o and a negative correlation with met_o. \
            (Obviously your decision about the person determines if you are a match or not.)',
            'I built and tested models to get the best fit and I got an accuracy score of 85.97% with the Logistic Regression model\
             and 75.71% accuracy when using the Naive Bayes model.',
            'From the model I was able to build a sort of equation which helps calculate if there is a match or not.'
           ]

questions_list = ['How many times have you met before?',
                  'How do you think they rate you? (On 10)',
                  "What's you initial decision about them? How much do you match? (0-1)",
                  "What percentage of their food do you think they'll share?",
                  'How ambitious do you think (s)he is? (Percentage)',
                  'How fun are they? On a scale of 1-10?',
                  "Rate how intelligent you think they are, on 10.",
                  "Are they malicious or sincere? To what extent? (Mid = 5)",
                  "Do you think them attractive? (0/1)",
                  'Rate how similar you think you are. Percentage ']

variables_list = [0]*10 #[0 for v in range(10)]
#-----------------------

def solve_match(input_on_submit, submit_value):
    #update variables
    variables_list[submit_value] = input_on_submit
    
    (met_o, prob_o, like_o, shar_o, amb_o, fun_o, intel_o, sinc_o, attr_o, dec_o) = map(float, variables_list)
    sinc_o = 0 if sinc_o < 5 else 1
    
    met_o *= 0.1
    prob_o *= 0.1
    shar_o *= 0.1
    like_o *= 0.1
    amb_o *= 0.01
    fun_o *= 0.1
    intel_o *= 0.1
    sinc_o *= 0.1
    attr_o *= 0.1
    dec_o *= 0.01
    
    expnum = np.exp(np.sum(variables_list))
    
    match = (0.06)*expnum - (0.01)*met_o + (0.26)*prob_o + (0.3)*like_o + (0.23)*shar_o + (0.13)*amb_o + \
            (0.27)*fun_o + (0.17)*intel_o + (0.17)*sinc_o + (0.27)*attr_o + (0.52)*dec_o
    return match

def print_formula(variables_list):
    formula = "(0.06)*{0} - (0.01)*{1} + (0.26)*{2} + (0.3)*{3} + (0.23)*{4} + (0.13)*{5} + (0.27)*{6} + (0.17)*{7} + (0.17)*{8} + (0.27)*{9} + (0.52)*{10}"\
                .format(np.exp(np.sum(variables_list)), variables_list[0] , variables_list[1],variables_list[2] , variables_list[3] , variables_list[4] ,variables_list[5] , variables_list[6] , variables_list[7] ,variables_list[8] , variables_list[9])
    #res = formula + ' = %s'%solve_match(input_on_submit, submit_value)
    
    return html.H5( formula  , style={'background-color':'grey',
                                      'text-align':'center'})

@app.callback(Output('make_graph','figure'),
              Input('input_on_submit', 'value'),
              Input('submit_val', 'n_clicks'),)
def make_graph(submit_value, input_on_submit): #...✅
    #calculate match score
    if submit_value > 0:
        score = solve_match(submit_value, input_on_submit)
    else:
        score = solve_match(0,0)
        
    #update image
    if score > 1:
            score = 1
    
    display(score)
    
    fig = go.Figure(go.Indicator(
                                mode = "gauge+number+delta",
                                value = score,
                                domain = {'x': [0, 1], 'y': [0, 1]},
                                title = {'text': "Match Results"},
                                delta = {'reference': .50},
                                gauge = {'axis': {'range': [None, 1]},
                                         'steps' : [
                                                     {'range': [0, 0.5], 'color': "gray"},
                                                     {'range': [0.5, .75], 'color': "lightgrey"}
                                                   ],
                                         'threshold' : {
                                                         'line': {
                                                                   'color': "red", 
                                                                   'width': 4
                                                                 }, 
                                                         'thickness': 0.75, 
                                                         'value': 0.85}}
                                )
                   );
    return fig

@app.callback(Output('curr_question','children'),
              [Input('submit_val', 'n_clicks')],)
              #State('input_on_submit', 'placeholder'))
def curr_question(n_clicks, questions = questions_list):
    while n_clicks < 9:
        return html.H3(questions_list[n_clicks], style={#'background-color':'green',
                                                   'min-height':'10vh',
                                                   'text-align':'center'})




In [5]:
app.layout = html.Div([
                        
                        html.Div([ html.Br(),
                                   dcc.Graph(#figure=make_graph(),
                                             id = 'make_graph', 
                                             style={'min-width':'40vh','min-height':'50vh','color':'green'}, 
                                             ),
                                  ], style={
                                            'border-width':'2em',
                                            'display':'table-cell',
                                            'margin':'4em',
                                            'padding':'4em',}), #✅
    
                        html.Div([
                            
                                    html.Div(
                                            [
                                                html.H3('Questions', style={'text-align':'right'}),
                                                html.Hr(),
                                                #Question
                                                html.Div(id='curr_question', style={'height':'fit-content'}),
                                                
                                                #Input
                                                dcc.Input(id='input_on_submit', 
                                                          max=10,
                                                          min=0,
                                                          placeholder=0,
                                                          required=True,
                                                          value=0,
                                                          step=0.2,
                                                          style={'width':'5vh',
                                                                 'margin-left':'15vh',
                                                                 'outline':'none'},
                                                          type='number'),
                                                
                                                html.Hr(),
                                                #Done Button
                                                html.Button('Next Question', id='submit_val', n_clicks=0, style={'border-radius':'10%',
                                                                                                                 'margin-left':'10vh',}),
                                                #Modify the graph
                                                html.Div(print_formula(variables_list))
                                        
                                             ],id='question-container',
                                              style={'width':'30vh', 'height':'fit-content'}
                                    ),
                                      
                                  ], style={'display':'table-cell', 
                                            'padding':'4em',
                                            #'background-color':'aqua',
                                            'margin':'4em',
                                            'min-height':'4em !important',
                                            'width':'30vh'}),
    
                        html.Div([html.H3('Define the Problem', style={'text-align':'right','color':'red'}),
                                  html.H5('Everybody goes on dates but what do you consider for the date to "go well"? '),
                                  
                                  html.H3('What Data was used? How was it cleaned?\
                                           How was it prepared? \
                                           What other operations were carried out on it?', style={'text-align':'right','color':'red'}),
                                  html.Ul([html.Li(x) for x in prep_list]),
                                  
                                  html.H3('How was the result achieved?', style={'text-align':'right','color':'red'}),
                                  html.Ul([html.Li(x) for x in mod_list]),
                                  
                                  html.H3('How can the result be useful?', style={'text-align':'right','color':'red'}),
                                  html.H5("It's a way to bring your doubts to rest when thinking of if to go on another date or not.")],
                                 style={'display':'table-cell', 
                                        'padding':'4em',
                                        #'background-color':'cyan',
                                        'font-family':'monospace',
                                        'margin':'4em',
                                        'min-height':'4em !important',
                                        'max-width':'30% !important'}) #✅
                      ], 
                       style={'color':'black',
                              'display':'flex',
                              'padding':'8em',
                              'position': 'fixed',
                             }
                     )

# --------------------------

In [6]:
#%tb
if __name__ == '__main__':
    app.run_server(debug=True,
                   width='fit-content')

Dash app running on http://127.0.0.1:8050/


In [7]:
dcc.Input?

Init signature:
dcc.Input(
    id=undefined,
    value=undefined,
    style=undefined,
    className=undefined,
    debounce=undefined,
    type=undefined,
    autoComplete=undefined,
    autoFocus=undefined,
    disabled=undefined,
    inputMode=undefined,
    list=undefined,
    max=undefined,
    maxLength=undefined,
    min=undefined,
    minLength=undefined,
    multiple=undefined,
    name=undefined,
    pattern=undefined,
    placeholder=undefined,
    readOnly=undefined,
    required=undefined,
    selectionDirection=undefined,
    selectionEnd=undefined,
    selectionStart=undefined,
    size=undefined,
    spellCheck=undefined,
    step=undefined,
    n_submit=undefined,
    n_submit_timestamp=undefined,
    n_blur=undefined,
    n_blur_timestamp=undefined,
    loading_state=undefined,
    persistence=undefined,
    persisted_props=undefined,
    persistence_type=undefined,
    **kwargs,
)
Docstring:     
An Input component.
A basic HTML input control for entering text, numbe